# First Passage Time Probabilities

In [2]:
import numpy as np
import pandas as pd

def first_passage_probabilities(i, j, P, n):
    # make sure states are labeled starting from 0
    # i is the starting state
    # j is the target state
    # P is the transition matrix
    # n is the number of steps to first passage
    F = []   # vector of first passage time probabilities
    PIJ = []   # vector of n-step transition probabilities from i to j
    PJJ = []   # vector of n-step transition probabilities from j to j

    # Convert state names to integer indices
    i_index = P.index.get_loc(i)
    j_index = P.index.get_loc(j)

    # First, get n-step probabilities
    for k in range(1, n+1):
        Q = np.linalg.matrix_power(P, k)
        Q = pd.DataFrame(Q)
        PJJ.append(Q.iloc[j_index, j_index])   # P^(k)jj
        PIJ.append(Q.iloc[i_index, j_index])   # P^(k)ij

    PJJ = pd.DataFrame(PJJ)
    PIJ = pd.DataFrame(PIJ)

    # Now, compute first passage probabilities
    for m in range(0, n):  # n is the number of steps in the path
        if n == 0:
            F.append(PIJ.iloc[0, 0])  # prob of going from i to j in one step
        else:
            summation = 0
            for k in range(1, m+1):
                summation += F[k-1] * PJJ.iloc[m-k, 0]  # there is only one column, column 0
            temp = PIJ.iloc[m, 0] - summation
            F.append(temp)

    index = ['1']
    for p in range(2, n+1):
        s = str(p)
        index.append(s)

    F = pd.DataFrame(F, index=index)
    F.rename(columns={0: f'f({i},{j})'}, inplace=True)

    return F

# Example
P = pd.DataFrame({'GM': [0.5, 0.2, 0.2, 0.1],
                  'Ford': [0.2, 0.4, 0.2, 0.2],
                  'Chrysler': [0.1, 0.1, 0.7, 0.1],
                  'Foreign': [0.1, 0.1, 0.2, 0.6]},
                 index=['GM', 'Ford', 'Chrysler', 'Foreign'])
P = P.T
i = 'GM'   # initial state
j = 'Ford'   # destination state
n = 10   # path length for first passage computation
F = first_passage_probabilities(i, j, P, n)

# Displaying output as a DataFrame
output_df = pd.DataFrame(F, columns=[f'f({i},{j})'])
output_df.index.name = 'Steps'
print(output_df)
print(" ")
print("Total probabilities:")
print(F.sum(axis=0))


       f(GM,Ford)
Steps            
1        0.200000
2        0.130000
3        0.095000
4        0.075400
5        0.062870
6        0.053833
7        0.046721
8        0.040819
9        0.035778
10       0.031408
 
Total probabilities:
f(GM,Ford)    0.77183
dtype: float64
